# Imports

In [1]:
import functools
import os
import time
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

from clu import metric_writers
import numpy as np
import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp
from jax.experimental import sparse
jax_device = jax.devices("gpu")[0]
jax.config.update("jax_default_device", jax_device)

import optax
import orbax.checkpoint as ocp

import h5py
import natsort
import tensorflow as tf
from scipy.ndimage import geometric_transform
from scipy.ndimage import gaussian_filter

In [2]:
from ISP_baseline.src import models, trainers, utils
from ISP_baseline.models import Uncompressed 

from swirl_dynamics import templates
from swirl_dynamics.lib import metrics
from pysteps.utils.spectral import rapsd

Pysteps configuration file found at: /share/data/willett-group/oortsang/miniconda/envs/jaxisp-env/lib/python3.11/site-packages/pysteps/pystepsrc



In [3]:
# For use with our MFISNet-style dataset
import os
from ISP_baseline.src.data_io import (
    load_hdf5_to_dict,
    load_cart_multifreq_dataset,
    load_single_dir_slice,
    load_multi_dir_slice,
    get_multifreq_dset_dirs,
)
from ISP_baseline.src.datasets import (
    convert_mfisnet_data_dict,
    setup_tf_dataset,
    get_io_mean_std,
)
from ISP_baseline.src.more_metrics import (
    l2_error,
)

In [4]:
# To avoid tf to use GPU memory
tf.config.set_visible_devices([], device_type='GPU')

W0000 00:00:1761175243.314895 1423549 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


## Load the dataset

In [5]:
rlc_repo = os.path.join("/", "home-nfs", "oortsang", "rlc-repo")
dataset_dir = os.path.join(rlc_repo, "dataset")

In [6]:
L = 4 # number of levels (even number)
s = 12 # leaf size for N_x = 192
# s = 5 # leaf size
# s = 6
# r = 3 # rank -- not used??

downsample_ratio = 1
s = s // downsample_ratio

# Discretization of Omega (n_eta * n_eta).
neta = (2**L)*s

# Number of sources/detectors (n_sc).
# Discretization of the domain of alpha in polar coordinates (n_theta * n_rho).
# For simplicity, these values are set equal (n_sc = n_theta = n_rho), facilitating computation.
nx = (2**L)*s

# Standard deviation for the Gaussian blur.
blur_sigma = 0.5

# Batch size.
batch_size = 16

# Number of training datapoints.
# NTRAIN = 21000
# NTRAIN = 2000 # Reduced for debugging purposes
NTRAIN = 1000 # for debugging purposes
NVAL   = 1000

In [7]:
# kbar_str_list = ["2.5", "5", "10"]
# kbar_str_list = ["2", "5", "10"] # uuhhh I haven't prepared the 2.5 data
kbar_str_list = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
nk = len(kbar_str_list)

In [8]:
train_dirs = get_multifreq_dset_dirs(
    "train",
    kbar_str_list,
    base_dir=dataset_dir,
    dir_fmt="{0}_measurements_nu_{1}"
)
train_mfisnet_dd = load_cart_multifreq_dataset(
    train_dirs,
    global_idx_start=0,
    global_idx_end=NTRAIN,
)
print(f"Loaded: {', '.join([f'{key}{val.shape}' for (key, val) in train_mfisnet_dd.items()])}")
train_wb_dd = convert_mfisnet_data_dict(
    train_mfisnet_dd,
    blur_sigma=blur_sigma,
    downsample_ratio=downsample_ratio,
)
# eta     = train_wb_dd["eta"]
# scatter = train_wb_dd["scatter"]

train_eta     = train_wb_dd["eta"]
train_scatter = train_wb_dd["scatter"]
# Get mean/std for each
(
    train_scatter_mean,
    train_scatter_std,
    train_eta_mean,
    train_eta_std
) = get_io_mean_std(train_scatter, train_eta)

Loaded: x_vals(192,), q_cart(1000, 192, 192), sample_completion(1000,), d_rs(1000, 10, 192, 192)


In [9]:
train_dataset, train_dloader = setup_tf_dataset(
    train_eta,
    train_scatter,
    batch_size=batch_size,
    repeats=True,
)

In [10]:
val_dirs = get_multifreq_dset_dirs(
    "val",
    kbar_str_list,
    base_dir=dataset_dir,
    dir_fmt="{0}_measurements_nu_{1}"
)
val_mfisnet_dd = load_cart_multifreq_dataset(
    val_dirs,
    global_idx_start=0,
    global_idx_end=NVAL,
)
print(f"Loaded: {', '.join([f'{key}{val.shape}' for (key, val) in val_mfisnet_dd.items()])}")
val_wb_dd = convert_mfisnet_data_dict(
    val_mfisnet_dd,
    blur_sigma=blur_sigma,
    downsample_ratio=downsample_ratio,
)
# Try downsampling since the sparsepolartocartesian step is so slow :((
val_eta     = val_wb_dd["eta"]
val_scatter = val_wb_dd["scatter"]

Loaded: x_vals(192,), q_cart(1000, 192, 192), sample_completion(1000,), d_rs(1000, 10, 192, 192)


In [11]:
val_dataset, val_dloader = setup_tf_dataset(
    val_eta,
    val_scatter,
    batch_size=16,
    repeats=False,
)

In [12]:
train_eta.shape, train_scatter.shape

((1000, 192, 192), (1000, 2, 36864, 10))

## Architecture

In [13]:
N_cnn_layers = 9
# N_cnn_channels = 6
N_cnn_channels = 20
# kernel_size = 3
kernel_size = 5
io_norm = True

In [14]:
# utils.load_mats_from_fp(os.path.join("tmp", "cart_and_rot_mats", f"mats_neta{neta}_nx{nx}.npz"))

In [15]:
%%time
from datetime import datetime
print(f"Starting at {datetime.now()}...")
cart_mat, r_index = utils.load_or_create_mats(
    neta,
    nx,
    mats_dir=os.path.join("tmp", "cart_and_rot_mats"),
    mats_format="mats_neta{0}_nx{1}.npz",
    save_if_created=True,
)

# cart_mat = utils.SparsePolarToCartesian(neta, nx)
# r_index  = utils.rotationindex(nx)

Starting at 2025-10-22 18:21:18.928378...
CPU times: user 61.5 ms, sys: 211 ms, total: 272 ms
Wall time: 275 ms


In [16]:
# import tornado, asyncio
# print(tornado.version)

In [17]:
# asyncio.__name__

In [18]:
core_module = Uncompressed.UncompressedModelFlexible(
    nx = nx,
    neta = neta,
    cart_mat = cart_mat,
    r_index = r_index,
    # New parameters
    nk=nk,
    N_cnn_layers=N_cnn_layers,
    N_cnn_channels=N_cnn_channels,
    kernel_size=kernel_size,
    # I/O Normalization?
    in_norm=io_norm,
    out_norm=io_norm,
    in_mean=jnp.array(train_scatter_mean),
    in_std=jnp.array(train_scatter_std),
    out_mean=jnp.array(train_eta_mean),
    out_std=jnp.array(train_eta_std),
)

# core_module = Uncompressed.UncompressedModel(
#     nx = nx,
#     neta = neta,
#     cart_mat = cart_mat,
#     r_index = r_index,
#     # # Doesn't support these
#     # nk=nk,
#     # N_cnn_layers=N_cnn_layers,
#     # N_cnn_channels=N_cnn_channels,
#     # kernel_size=kernel_size,
# )

In [19]:
Model = models.DeterministicModel(
    input_shape = train_scatter[0].shape,
    core_module = core_module
)

In [20]:
rng = jax.random.PRNGKey(888)
params = Model.initialize(rng)
param_count = sum(x.size for x in jax.tree_util.tree_leaves(params))
print('Number of trainable parameters:', param_count)

2025-10-22 18:21:21.977662: W external/xla/xla/stream_executor/cuda/subprocess_compilation.cc:237] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2025-10-22 18:21:21.977673: W external/xla/xla/stream_executor/cuda/subprocess_compilation.cc:240] Used ptxas at /usr/local/cuda/bin/ptxas
2025-10-22 18:21:21.977713: W external/xla/xla/stream_executor/gpu/redzone_allocator_kernel_cuda.cc:135] UNIMPLEMENTED: /usr/local/cuda/bin/ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


Number of trainable parameters: 3374411


In [21]:
train_scatter[0].shape

(2, 36864, 10)

In [22]:
type(cart_mat)

jax.experimental.sparse.bcoo.BCOO

In [23]:
type(r_index)

jaxlib.xla_extension.ArrayImpl

## Training

In [24]:
epochs = 500
num_train_steps = NTRAIN * epochs // 16  #@param
workdir = os.path.abspath('') + f"/tmp/2025-10-19_Uncompressed_nx_{nx}_nk_{nk}"  #@param
if os.path.exists(workdir):
    import shutil
    shutil.rmtree(workdir)

initial_lr = 1e-5 #@param
peak_lr = 5e-3 #@pawram
warmup_steps = num_train_steps // 20  #@param
end_lr = 1e-8 #@param
ckpt_interval = 2000  #@param
max_ckpt_to_keep = 3  #@param

In [25]:
trainer = trainers.DeterministicTrainer(
    model=Model,
    rng=jax.random.PRNGKey(42),
    optimizer=optax.adam(
        learning_rate=optax.warmup_cosine_decay_schedule(
            init_value=initial_lr,
            peak_value=peak_lr,
            warmup_steps=warmup_steps,
            decay_steps=num_train_steps,
            end_value=end_lr,
        ),
    ),
)

In [26]:
untrained_inference_fn = trainers.DeterministicTrainer.build_inference_fn(
    trainer.train_state, core_module
)

In [27]:
print(f"Memory usage associated with a single forward pass...")
_ = utils.get_memory_info_jax(device=jax_device, print_msg=True)
# pre-run:     244
# after 16:   2694
# after 32:   5286
# after 64:  10469
# after 128: 20865
trial_batch_size = 16

# first_batch = next(train_dloader)
trial_input = train_wb_dd["scatter"][:trial_batch_size]
trial_output = untrained_inference_fn(trial_input)

_ = utils.get_memory_info_jax(device=jax_device, print_msg=True)

# Okay, so running the forward pass is not a problem

Memory usage associated with a single forward pass...
RAM Used (MB): 68189
VRAM (MB): 87439 free of 87532 total (within preallocation); usage is currently 92 and peaked at 297
RAM Used (MB): 68328
VRAM (MB): 87437 free of 87532 total (within preallocation); usage is currently 95 and peaked at 3136


In [28]:
# asldkfjsdlkjfiowf

In [29]:
_ = utils.get_memory_info_jax(device=jax_device, print_msg=True)

try:
    eval_dloader = train_dloader
    templates.run_train(
        train_dataloader=train_dloader,
        trainer=trainer,
        workdir=workdir,
        total_train_steps=num_train_steps,
        metric_writer=metric_writers.create_default_writer(
            workdir, asynchronous=False
        ),
        metric_aggregation_steps=10,
        eval_dataloader=eval_dloader,
        eval_every_steps = 100,
        num_batches_per_eval = 1,
        callbacks=(
            templates.TqdmProgressBar(
                total_train_steps=num_train_steps,
                train_monitors=("train_loss",),
                eval_monitors=("eval_rrmse_mean", "eval_rel_l2_mean"),
            ),
            templates.TrainStateCheckpoint(
                base_dir=workdir,
                options=ocp.CheckpointManagerOptions(
                    save_interval_steps=ckpt_interval, max_to_keep=max_ckpt_to_keep
                ),
            ),
        ),
    )
except:
    # Print out memory info on failure...
    _ = utils.get_memory_info_jax(device=jax_device, print_msg=True)


_ = utils.get_memory_info_jax(device=jax_device, print_msg=True)

  0%|          | 0/31250 [00:00<?, ?step/s]

2025-10-22 18:22:23.239677: E external/xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng0{} for conv %cudnn-conv-bw-filter.14 = (f32[20,5,5,90]{3,2,1,0}, u8[0]{0}) custom-call(%bitcast.35574, %bitcast.35576), window={size=5x5 pad=2_2x2_2}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convBackwardFilter", metadata={op_name="jit(_train_step)/jit(main)/transpose(jvp(UncompressedModelFlexible))/convs_4/conv_general_dilated" source_file="/share/data/willett-group/oortsang/ISP_baseline_fork/ISP_baseline/models/Uncompressed.py" source_line=200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2025-10-22 18:22:23.369583: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 1.130012359s
Trying algorithm eng0{} for conv %cudnn-conv-bw-filter.14 = (f32[20,5

RAM Used (MB): 195750
VRAM (MB): 87350 free of 87532 total (within preallocation); usage is currently 182 and peaked at 40887


## Inference

In [30]:
trained_state = trainers.TrainState.restore_from_orbax_ckpt(
    f"{workdir}/checkpoints", step=None
)

inference_fn = trainers.DeterministicTrainer.build_inference_fn(
    trained_state, core_module
)

In [31]:
NTEST = 1000

test_dirs = get_multifreq_dset_dirs(
    "test",
    kbar_str_list,
    base_dir=dataset_dir,
    dir_fmt="{0}_measurements_nu_{1}"
)

test_mfisnet_dd = load_cart_multifreq_dataset(
    test_dirs,
    global_idx_start=0,
    global_idx_end=NTEST,
)
print(f"Loaded: {', '.join([f'{key}{val.shape}' for (key, val) in test_mfisnet_dd.items()])}")
test_wb_dd = convert_mfisnet_data_dict(
    test_mfisnet_dd, 
    blur_sigma=blur_sigma,
    downsample_ratio=downsample_ratio
)

# Try downsampling since the sparsepolartocartesian step is so slow :((
test_eta     = test_wb_dd["eta"] # [..., ::2, ::2]
test_scatter = test_wb_dd["scatter"] # [..., ::2, ::2, :]


KeyboardInterrupt



In [ ]:
test_batch_size = 20
test_dataset, test_dloader = setup_tf_dataset(
    test_eta,
    test_scatter,
    # val_eta,
    # val_scatter,
    batch_size=test_batch_size,
)

In [ ]:
validation_errors_rrmse = []
validation_errors_rel_l2 = []
validation_errors_rapsd = []
pred_eta = np.zeros(test_eta.shape)

rrmse = functools.partial(
    metrics.mean_squared_error,
    sum_axes=(-1, -2),
    relative=True,
    squared=False,
)
rel_l2 = functools.partial(
    l2_error,
    l2_axes=(-1, -2),
    relative=True,
    squared=False,
)

# for b, batch in enumerate(test_dloader):
for b, batch in enumerate(val_dloader):
    # pred = inference_fn(batch[0])
    pred = inference_fn(batch["scatter"])
    batch_slice = np.s_[b*test_batch_size: (b+1)*test_batch_size, :, :]
    pred_eta[batch_slice] = pred
    true = batch["eta"]
    validation_errors_rrmse.append(rrmse(pred=pred, true=true))
    validation_errors_rel_l2.append(rel_l2(pred=pred, true=true))
    for i in range(true.shape[0]):
        validation_errors_rapsd.append(np.abs(np.log(rapsd(pred[i],fft_method=np.fft)/rapsd(true[i],fft_method=np.fft))))

print(f"Mean rel l2 error: {np.mean(validation_errors_rel_l2)*100:.3f}%")
print('relative root-mean-square error = %.3f' % (np.mean(validation_errors_rrmse)*100), '%') 
print('mean energy log ratio = %.3f' % np.mean(validation_errors_rapsd)) 

In [ ]:
# test_data_path = os.path.abspath('../..') + '/data/10hsquares_testdata'
# test_data_path = os.path.join("data", "testdata")

# with h5py.File(f'{test_data_path}/eta.h5', 'r') as f:
#     # Read eta data, apply Gaussian blur, and reshape
#     eta_re = f[list(f.keys())[0]][:, :].reshape(-1, neta, neta)
#     blur_fn = lambda x: gaussian_filter(x, sigma=blur_sigma)
#     eta_test = np.stack([blur_fn(img.T) for img in eta_re]).astype('float32')
    
# # Loading and preprocessing scatter data (Lambda)
# # with h5py.File(f'{test_data_path}/scatter_order_8.h5', 'r') as f:
# with h5py.File(f'{test_data_path}/scatter.h5', 'r') as f:
#     keys = natsort.natsorted(f.keys())

#     # Process real part of scatter data
#     tmp1 = f[keys[3]][:, :]
#     tmp2 = f[keys[4]][:, :]
#     tmp3 = f[keys[5]][:, :]
#     scatter_re = np.stack((tmp1, tmp2, tmp3), axis=-1)

#     # Process imaginary part of scatter data
#     tmp1 = f[keys[0]][:, :]
#     tmp2 = f[keys[1]][:, :]
#     tmp3 = f[keys[2]][:, :]
#     scatter_im = np.stack((tmp1, tmp2, tmp3), axis=-1)
    
#     # Combine real and imaginary parts
#     scatter_test = np.stack((scatter_re, scatter_im), axis=1).astype('float32')
    
# # Clean up temporary variables to free memory
# del scatter_re, scatter_im, tmp1, tmp2, tmp3
